## Load necessary python libraries

In [10]:
import os
import napari
import tifffile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import bigfish
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.multistack as multistack
from skimage.io import imread
import bigfish.plot as plot
from napari.types import ImageData, LabelsData

from matplotlib.backends.backend_qt5agg import FigureCanvas
from dask.array.image import imread as imr
from imaris_ims_file_reader.ims import ims


from skimage.measure import label, regionprops
import bigfish.segmentation as segmentation
from skimage.segmentation import find_boundaries
from skimage.morphology.selem import disk
from pathlib import Path
from napari.layers import Image
from magicgui import magicgui, magic_factory, widgets
from magicgui.widgets import Table

print("Big-FISH version: {0}".format(bigfish.__version__))
print("numpy version: {0}".format(np.__version__))

print('Average measured size from image :',(123.4/1024)*1*1000) #objectRad in nm

Big-FISH version: 0.6.2
numpy version: 1.23.0
Average measured size from image : 120.5078125


In [11]:
mIdentifier = 'blur_'

# Run GUI

In [12]:
VOXEL_RADIUS = (0,0,0)
OBJECT_RADIUS = (0,0,0)

DEFAULT_CHOICES=['None']
SET_TABLE = 0
CELL_CHOICE = 0
DEBUG = False
@magicgui(auto_call=True,    
    main_window=False,
          persist=False,
    base_folder={"label": "Choose home folder:",
                 'mode': 'd'},
    dropdown=dict(
            widget_type="Select", choices=DEFAULT_CHOICES, label="Choose cell to analyse"
        ),
    layout="vertical",
         )
def choose_home_folder(dropdown, base_folder = Path.home()):
    if DEBUG==True:    
        print("calling choose home folder")
        print('dropdown :', dropdown)
        print('bf: ', base_folder)

    return base_folder, dropdown

@choose_home_folder.changed.connect
def choose_cell_on_file_change(event=None):
    global SET_TABLE, CELL_CHOICE
    if DEBUG==True: 
        print('calling populate cell choices')
        print('choose home folder count :', choose_home_folder.call_count)
    if choose_home_folder.call_count == 1 and SET_TABLE==0:
        SET_TABLE = 1
        if DEBUG==True: print('value of set table set to : ', SET_TABLE)
    hm = choose_home_folder.base_folder.value
    cellChoices = [i for i in os.listdir(hm) if mIdentifier in i and os.path.isfile(os.path.join(hm, i))]
    if cellChoices is not None:
        choose_home_folder.dropdown.choices = cellChoices
        global DEFAULT_CHOICES
        DEFAULT_CHOICES = deepcopy(cellChoices)
        choose_home_folder.dropdown.value = cellChoices[CELL_CHOICE]
        


@choose_home_folder.called.connect
def on_dropdown_changed(value: str):
    def catchval(new_value):
        thresholdTable[new_value['column_header']][new_value['row_header']] = new_value['data']
        table.to_dataframe().to_csv(str(value[0])+'/'+'thresholds.csv')
    
    if DEBUG==True: 
        print('calling on drowpdown changed')
        
    global SET_TABLE, CELL_CHOICE, DEFAULT_CHOICES
    if np.size(value[1])!=0:
        if DEBUG==True: print('chosen cell', str(value[1][0]))
        CELL_CHOICE = DEFAULT_CHOICES.index(value[1][0])
        getFolderName.folder_name.value = os.path.join(str(value[0]), str(value[1][0]))
        if SET_TABLE==1:
            optionNames = [i.split(mIdentifier)[-1].split('.tif')[0] for i in DEFAULT_CHOICES]
            hm = str(choose_home_folder.base_folder.value.parent)
            if os.path.exists(hm+'/'+'thresholds.csv'):
                df = pd.read_csv(hm+'/'+'thresholds.csv', index_col=0)
                thresholdTable = df.to_dict()
                optionsIncsv  = list(df['cell'])
                extras = list(set(optionNames).symmetric_difference(set(optionsIncsv)))
                if np.size(extras)!=0:
                    print('Missing files or threshold!')

            else:
                print('Threshold file not found!!')       
            table = Table(value=thresholdTable)
            table.changed.connect(catchval)
            tablewidget = viewer.window.add_dock_widget(table, tabify=False, name = 'threshold table')
            SET_TABLE = 0

    else:
        print('no cells found')
        getFolderName.folder_name.value = str(value[0])   
        
        
@magicgui(
    auto_call=True,
    layout="vertical",
)
def getFolderName(
    folder_name="cell folder",    
    ):
    if DEBUG==True:
        print("calling getFolderName")
    if not os.path.exists(folder_name):
        folder_name = Path.home()
    return folder_name

@getFolderName.called.connect
def _on_func_getFolderName(value: str):
    showMovie.input.value = value
    

@magicgui(auto_call=False, input={"visible": False, "label": " "}, persist=True)
def showMovie(input: str)-> "napari.types.LayerDataTuple":
    """Callable function B."""
    
    if DEBUG==True: print("calling showMovie\n"); print(input)
    movieName = input
    if mIdentifier in movieName and os.path.isfile(movieName):
        print(movieName)
        print('image found')
        stackCell0 = imr(movieName)[0]#+'/*.tif'
        stackCell = np.array(np.max(stackCell0, axis=0))
        metadata = {'name': 'stackCell', 'colormap': 'green'}
        metadata2 = {'name': 'full', 'colormap': 'green'}
        layer_type = 'image'
        viewer = napari.current_viewer()
        image_layer = viewer.add_image(stackCell,
                         metadata=metadata)
        image_layer2 = viewer.add_image(stackCell0,
                         metadata=metadata2)

        return (stackCell, metadata, layer_type)
@magicgui(auto_call=True)
def process_image(image_input: ImageData, threshold: int = 10000, kernel_size_dilation: int=10, kernel_size_erosion: int=3, smoothness: int=4) -> LabelsData:
    if input is not None:
        saveMaskPath = os.path.join(str(choose_home_folder.base_folder.value.parent),'masks_tx')
        nameKey = getFolderName.folder_name.value.split('/')[-1].split('.tif')[0]
        maskPathtx = os.path.join(saveMaskPath,nameKey+'.npy')
#         print('maskpath :', maskPathtx)
        if not os.path.exists(maskPathtx):
            # process the image
            labeled_image = segmentation.thresholding(np.asarray(image_input), threshold=threshold)
            labeled_image_new = segmentation.clean_segmentation(labeled_image, small_object_size=1, fill_holes=True)
            labeled_image_new = stack.erosion_filter(labeled_image_new, kernel_shape='disk', kernel_size=1)
            labeled_image_new = stack.dilation_filter(labeled_image_new, kernel_shape='disk', kernel_size=kernel_size_dilation)
            labeled_image_new = stack.erosion_filter(labeled_image_new, kernel_shape='disk', kernel_size=kernel_size_erosion)
            labeled_image_labeled = segmentation.label_instances(labeled_image_new)
            labeled_image_labeled_final = segmentation.clean_segmentation(labeled_image_labeled, smoothness=smoothness, delimit_instance=False)
            return labeled_image_labeled_final
        else:
            print('label found')
            tx_label_file = np.load(maskPathtx)
#             Tx_mask_new = np.array(label(tx_label_file),dtype=bool)
            return tx_label_file


    
@magicgui(main_window=False,
         call_button="Save Mask"
         )
def saveMask(folder_name="cell folder",):
    if DEBUG==True: print('basefolder = ',choose_home_folder.base_folder.value.parent)
    saveMaskPath = os.path.join(choose_home_folder.base_folder.value.parent,'masks_tx')
    nameKey = Path(getFolderName.folder_name.value).name.split('.tif')[0]
    if not os.path.exists(saveMaskPath):
        os.makedirs(saveMaskPath)
    folder_name = saveMaskPath
    label_clean = viewer.layers['process_image result'].data
    maskPathtx = os.path.join(saveMaskPath,nameKey)
    print(nameKey)
    print(saveMaskPath)
    if DEBUG==True: print(maskPathtx)
    stack.save_array(label_clean, maskPathtx)
    


@magicgui(main_window=False,
         call_button="Analyse new cell",
         )
def loadNewCell():
    viewer.layers.select_all()
    viewer.layers.remove_selected()

@magicgui(main_window=False,
         call_button="Analyse new movie",)
def resetParametersForNewMovie():
    global CELL_CHOICE
    viewer.layers.select_all()
    viewer.layers.remove_selected()
    choose_home_folder.reset_call_count()
    CELL_CHOICE=0
    choose_home_folder.dropdown.value = DEFAULT_CHOICES[CELL_CHOICE]    
    

container = widgets.Container(
    widgets=[choose_home_folder, 
             getFolderName,
             showMovie,
             process_image,
             saveMask, 
             loadNewCell,
             resetParametersForNewMovie], layout="vertical", labels=False
)
container.native.setMinimumWidth(500)

viewer = napari.Viewer()
viewer.window.add_dock_widget(container, area="right", name = 'Draw masks')

no cells found
Missing files or threshold!
C:\Users\uid-1204\Desktop\test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\blurs\blur_cell_5.tif
image found
no cells found
no cells found
no cells found
no cells found
blur_cell_5
C:\Users\uid-1204\Desktop\test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\masks_tx
